# Model Accuracy

Up until now, I've been using mostly qualitative, visual assessment to determine how well a model performed. I want to create a better metric. I'm thinking that a good metric will be the percentage of pixels correctly labelled; let's see how well it works.

In [1]:
import torch
import pandas as pd

import monai
from monai.transforms import (
    Compose,
    LoadNiftid,
    NormalizeIntensityd,
    AddChanneld,
    ToTensord,
    CenterSpatialCropd,
)

import numpy as np

In [2]:
def get_model_accuracy(model):
    data_df = pd.read_csv('/data/shared/prostate/yale_prostate/input_lists/MR_yale.csv')

    test_imgs = data_df['IMAGE'][295:].tolist()
    test_masks = data_df['SEGM'][295:].tolist()

    test_dicts = [{'image': image, 'mask': mask} for (image, mask) in zip(test_imgs, test_masks)]

    # Basic transforms
    data_keys = ["image", "mask"]
    data_transforms = Compose(
        [
            LoadNiftid(keys=data_keys),
            AddChanneld(keys=data_keys),
            NormalizeIntensityd(keys="image"),
            CenterSpatialCropd(
                keys=data_keys,
                roi_size=[128, 128, 16]
            ),
        ]
    )

    test_dataset = monai.data.Dataset(
        data=test_dicts,
        transform=Compose(
            [
                data_transforms,
                ToTensord(keys=data_keys)
            ]
        ),
    )
        
    accuracies = np.array([])
    for sample in test_dataset:
        test_image = sample['image'].unsqueeze(0)
        test_mask = model(test_image)
        test_mask = test_mask.argmax(1).detach()
        
        accuracy = 1 - torch.sum(torch.abs(sample['mask'] - test_mask)) / (256 * 256 * 16)
        accuracies = np.append(accuracies, accuracy)
        
    avg_accuracy = np.average(accuracies)
    median_accuracy = np.median(accuracies)
    return {
        'average': avg_accuracy,
        'median': median_accuracy
    }

In [3]:
import glob
import CustomModels

In [4]:
def print_acc_metrics(model_class, checkpoint_path):
    model = model_class.load_from_checkpoint(checkpoint_path)
    print(model.hparams.name)
    accuracy = get_model_accuracy(model)
    print('Average Accuracy: ', accuracy['average'])
    print('Median Accuracy: ', accuracy['median'])

## 3D UNet

## Dice + Focal Loss

In [5]:
print_acc_metrics(CustomModels.UNet_DF, 'models/7-24-2020_dicefocal_prostate/_ckpt_epoch_19.ckpt')

7-24-2020_dicefocal_prostate
Average Accuracy:  0.9957978950356537
Median Accuracy:  0.9966726303100586


## MaskGAN

In [7]:
print_acc_metrics(CustomModels.MaskGAN, 'models/7-23-2020_MaskGAN_prostate/_ckpt_epoch_1964.ckpt')

7-23-2020_MaskGAN_prostate
Average Accuracy:  0.9964361190795898
Median Accuracy:  0.9968929290771484
